In [1]:
import pandas as pd

In [2]:
interactions = pd.read_csv("./data/interactions.csv", names=['datetime', 'viewer_id', 'viewer_name', 'profile_id','profile_name', 'status', 'score'])

interactions['viewer_id'] = interactions['viewer_id'].apply(lambda w: w.split("-")[-1])

update = interactions.groupby('viewer_id')['profile_id'].apply(list).reset_index()
update

In [3]:
interactions['viewer_id'] = interactions['viewer_id'].apply(lambda w: w.split("-")[-1])

In [5]:
query_id = 'f8476bcc'

,viewer_id,profile_id
0,309e9139,"[309e9139, ed84533f, 33ddcb8b, 76672d66]"
1,7faa872c,"[d6d104b4, 55628e45, e60ff7f9, 7faa872c]"
2,Default,"[afc9280f, 9c1b751e, a2ab61b3, 973a2b41, afc92..."
3,f8476bcc,"[6f8db1aa, a585683b]"
4,fde9c9f3,"[050a9be7, 229998fb, 836066e3, 71c74a49, 5ccd5..."
5,ssse1024,"[e288e6d9, 9dfde2cb, 9c593909, 9dfde2cb, 5020e..."


In [7]:
update.loc[update['viewer_id'] == query_id, :]

,viewer_id,profile_id
3,f8476bcc,"[6f8db1aa, a585683b]"


In [9]:
def update_embedding(query_id):
    interactions = pd.read_csv("/Users/sudhirsingh/PyCharmProjects/story/src/services/frontend/data/interactions.csv", names=['datetime', 'viewer_id', 'viewer_name', 'profile_id','profile_name', 'status', 'score'])
    interactions['viewer_id'] = interactions['viewer_id'].apply(lambda w: w.split("-")[-1])
    update = interactions.groupby('viewer_id')['profile_id'].apply(list).reset_index()

    need_to_update = update.loc[update['viewer_id'] == query_id, 'profile_id'].to_list()
    if need_to_update:
        import re
        import numpy as np
        from numpy.linalg import norm

        def str_to_array(s):
            if s is None:
                return None
            s = s.strip()
            if s.startswith('[') and s.endswith(']'):
                s = s[1:-1]
            s = re.sub(r'[\r\n]+',' ', s)
            s = re.sub(r'\s+', ' ', s).strip()
            return np.fromstring(s, sep=' ').astype(float)

        def avg(l):

            es = p[p['id'].isin(l)]['embedding'].values
            avg = np.mean(es, axis=0)
            n = avg / norm(avg)
            return n

        p = pd.read_csv("./data/profile_embedding.csv")
        p['embedding'] = p['embedding'].apply(str_to_array)

        update['updated_embedding'] = update['profile_id'].apply(avg)

        update = update.rename(columns={'viewer_id': 'id'})
        update = update.rename(columns={'updated_embedding': 'embedding'})
        update = update[['id', 'embedding']]
        update_dict = dict(zip(update['id'], update['embedding']))
        p['embedding'] = p.apply(
            lambda row: update_dict.get(row['id'], row['embedding']),
            axis=1
        )
        p.to_csv("/Users/sudhirsingh/PyCharmProjects/story/src/services/frontend/data/profile_embedding.csv", index=False)
        print('updated')
    else:
        ...

In [10]:
update_embedding('ssse1024')

updated
